In [14]:
import pandas as pd
import numpy as np
import random
import copy
import math
from collections import defaultdict

randomlist = []
demand = {}
random.seed(2) # Use the same random seed
for i in range(0,20):   # 20 depots
    n = random.randint(100,150) # Depot Capacity
    randomlist.append(n)

for i in range(20):
    demand["D"+str(i+1)] = randomlist[i]

sum = 0
for (k,v) in demand.items():
    sum = sum + v

supply = {}
for i in range(sum):
    supply[i] = 1.0

np.random.seed(2) # Use the same random seed
df_dkm = pd.DataFrame(np.random.uniform(0.5,20, size=(sum,20))).astype(float)

random.seed(2) # Use the same random seed
age_b = []
for i in range(sum):
    n = random.randint(1,3) #Only to generate integer values inclusive of 1 & 3
    age_b.append(n)

age = {}
for i in range(len(age_b)):
    age[i] = age_b[i]

kpl = []
co2 = []
random.seed(2) # Use the same random seed
for (k,v) in age.items():
    if (v==1):
        n = random.uniform(5.1,6)
        m = round(random.uniform(515, 524), 2)
        kpl.append(n)
        co2.append(m)
    if (v==2):
        n = random.uniform(4.1,5)
        m = round(random.uniform(525, 534), 2)
        kpl.append(n)
        co2.append(m)
    if (v==3):
        n = random.uniform(3.1,4)
        m = round(random.uniform(535, 540), 2)
        kpl.append(n)
        co2.append(m)

fc_kpl = {}
for i in range(sum):
    fc_kpl[i] =(111/ kpl[i])

co2_c = {}
for i in range(sum):
    co2_c[i] =(0.00118642*co2[i])

random.seed(2) # Use the same random seed
doc = {}
for i in range(0,20):
    n = random.uniform(50,100)
    doc[i] = n

df_tdkom = pd.DataFrame(np.zeros(df_dkm.shape))
for i in range(len(df_dkm.columns)):
    for j in range(len(df_dkm.index)):
        df_tdkom[i][j] = df_dkm[i][j]*fc_kpl[j] + df_dkm[i][j]*co2_c[j] + doc[i]
#df_tdkom
df_tdkom.columns = [("D"+str(i)) for i in range(1,21)]
dk = df_tdkom.transpose()
costs = dk.to_dict()
cols = sorted(demand.keys())

In [15]:
# mi = 10000
# print(supply['S1'])
# break
costs1=copy.deepcopy(costs)

costs2=copy.deepcopy(costs)
costs3=copy.deepcopy(costs)
for i in supply:
    mi=min(costs[i].values())
    # print(costs[i])
    # print(mi)
    for j in costs2[i]:
        costs2[i][j]-=mi
# print(costs2)
for i in demand :
    mi=10000
    for j in supply:
        if costs[j][i]<mi :
            mi=costs[j][i]
    for j in supply:
        costs3[j][i]=costs3[j][i]-mi
# print(costs3)

for i in demand:
    for j in supply:
        costs[j][i]= costs2[j][i]+costs3[j][i]


res = dict((k, defaultdict(int)) for k in costs)
g = {}
for x in supply:
    g[x] = sorted(costs[x].keys(), key=lambda g: costs[x][g])
for x in demand:
    g[x] = sorted(costs.keys(), key=lambda g: costs[g][x])
# print(costs1)
flag=0
while g:

    d = {}
    # print(demand,supply)

    for x in demand:
        d[x] = (costs[g[x][1]][x] - costs[g[x][0]][x])* min(demand[x],supply[g[x][0]]) if len(g[x]) > 1 else (costs[g[x][0]][x])*min(demand[x],supply[g[x][0]])
    s = {}
    for x in supply:
        d[x] = (costs[x][g[x][1]] - costs[x][g[x][0]])*min(supply[x],demand[g[x][0]]) if len(g[x]) > 1 else costs[x][g[x][0]]*min(supply[x],demand[g[x][0]])

    for x in supply :
      for y in demand:
        if costs[x][y]*min(supply[x],demand[y])<mi :
          mi= costs[x][y]*min(supply[x],demand[y])
    sd = max(d, key=lambda n: d[n])
    # t = max(s, key=lambda n: s[n])
    t,f="D","S"
    if demand.get(sd)!=None:
      t,f = sd, g[sd][0]
    else :
      t,f = g[sd][0],sd




    # t,f=tfin,fin
    # print(t,f)
    v = min(supply[f], demand[t])
    res[f][t] += v
    demand[t] -= v

    if demand[t] == 0:
        for k, n in supply.items():
            if n != 0:
                g[k].remove(t)
        del g[t]
        del demand[t]
    supply[f] -= v
    if supply[f] == 0:
        for k, n in demand.items():
            if n != 0:
                g[k].remove(f)
        del g[f]
        del supply[f]


# print(costs1)
cost = 0
for g in sorted(costs1):
    # print (g, " ",)
    for n in cols:
        y = res[g][n]
        if y != 0:
            pass
            # print (y,)
        cost += y * costs1[g][n]
        # print ("  ",)
    # print(" ")
# print ("Total Cost = ", cost)


In [16]:
df_tdkom

,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15,D16,D17,D18,D19,D20
0,271.033760,116.742341,268.714152,227.223283,259.142814,220.380876,169.904294,307.414469,202.366649,190.090822,321.766900,266.105313,131.657500,272.022272,164.984899,404.065347,427.551827,272.296584,399.542681,102.921519
1,327.224761,136.685737,248.921555,107.040246,157.807947,355.762965,192.238461,122.704215,186.583388,242.598412,292.295499,156.183325,359.366243,289.516162,315.569602,278.017546,451.521247,338.938633,153.460882,377.323179
2,476.000191,298.160293,402.368358,194.514522,318.186002,259.890800,260.093207,371.802821,294.661753,454.410182,296.710991,99.068302,221.249416,404.447753,251.118592,119.924224,201.671223,112.647729,461.632518,443.913348
3,488.016004,393.847749,426.364312,146.312542,242.334867,346.520064,264.096155,99.103518,556.924300,300.982572,329.300937,222.898006,206.370258,264.585875,491.369917,459.947963,288.750795,95.473844,461.108842,202.805353
4,338.366542,117.635435,324.837752,217.631075,298.484536,261.014455,232.380009,293.433002,475.178334,135.077718,213.079336,84.588051,373.676356,339.832242,181.168980,274.687575,362.219391,348.934036,149.798392,422.204655
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2484,406.985309,281.024639,399.321919,133.721453,142.825695,175.987239,157.553855,688.497044,693.782749,359.290042,443.884294,83.703288,695.349411,621.881961,224.173690,598.554134,574.670153,198.144554,363.760653,83.771481
2485,523.830374,273.960959,628.448381,447.762151,238.055544,492.070175,410.636545,84.028798,300.540501,244.221269,334.196897,553.357260,634.527149,299.172834,627.367027,616.288453,483.466009,187.232776,351.147773,84.441214
2486,317.200562,162.443317,445.491209,148.470430,430.245226,343.678450,326.242257,340.160168,455.906437,313.673799,444.875407,324.403048,439.681520,309.532766,196.903890,375.168053,394.003550,254.175169,450.464828,367.409195
2487,217.593107,201.358773,372.390276,164.142922,134.269322,124.957943,441.368116,121.562919,269.148388,418.781048,140.935016,144.650893,345.927036,182.749547,113.130728,122.281181,262.683791,358.580687,341.517571,436.915653


In [17]:
cost

287819.1977879968

stop = timeit.default_timer()
execution_time = stop - start

print("Program Executed in "+str(execution_time))